In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import string
import pandas as pd

#Snorkel
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function

pd.set_option('display.max_colwidth', None)

### Loading and Preprocessing Textual Data From Twitter and News Headlines

#### Loading DataFrames

In [2]:
# Twitter
df_twitter = pd.read_pickle('../crawlers/twitter/twitter_df.pkl')
df_twitter.rename(columns={'text':'title'}, inplace=True)
df_twitter = df_twitter[['title']]
df_twitter['source'] = 'Twitter'
df_twitter.index = df_twitter.index.rename('date')
df_twitter.sample(5)

,title,source
date,,
2022-04-28 00:53:21,Petrobras aprova venda de participação na Deten para a Cepsa por R$ 585 milhões https://t.co/hus5EDs3FO,Twitter
2022-03-30 10:00:23,"E agora, Nubank (NUBR33)? Revolut, maior concorrente global, chega ao Brasil - https://t.co/9I0B0LdUA6",Twitter
2022-03-11 06:22:48,"Quem está ajudando na independência financeira feminina? Ouça no podcast 'Pod isso, Meninas?'\nhttps://t.co/NYBFcZiiAD",Twitter
2022-03-25 09:41:26,Suzano é top pick do BofA entre commodities: ‘rali ainda não acabou’ - https://t.co/KrMAngvRPb,Twitter
2021-11-01 12:47:38,BRADESPAR - Press-release - Boletim Quinzenal Bradespar - #brap4 https://t.co/45pLwzvhuI,Twitter


In [3]:
# News Headlines
df_news = pd.read_pickle('../crawlers/suno/news_df.pkl')
df_news = df_news[['title']]
df_news['source'] = 'Suno'
df_news.sample(5)

,title,source
date,,
2019-04-02 21:05:00,Banco do Brasil vai vender participação da Neoenergia,Suno
2021-10-19 13:45:00,"Preço da gasolina chega a R$ 7,49 e gás de cozinha alcança R$ 135, diz ANP",Suno
2020-08-05 21:27:00,"BB, Itaú (ITB4) e Bradesco (BBDC4) reduzem juros de financiamentos",Suno
2022-02-10 19:52:00,"Vale (VALE3): produção de minério de ferro sobe 5,1% em 2021 e vendas crescem 9%",Suno
2020-08-01 12:13:00,André Brandão aceita convite para presidir o Banco do Brasil (BBAS3),Suno


In [4]:
# Append all dataframes

df_full = pd.DataFrame()

frames = [df_twitter, df_news]

for df in frames:
    df_full = df_full.append(df)

df_full.sample(5)

,title,source
date,,
2022-01-13 20:12:38,"BTG Pactual (BPAC11) passa a deter fatia total de 73,02% do Banco Pan (BPAN4) - https://t.co/zxTEoY8n7Q",Twitter
2022-03-17 13:54:55,MRV (MRVE3): resultado evidencia “dois mundos” da construtora e ações caem; empresa destaca desafios no Casa Verde e Amarela https://t.co/LtgP8COR3H,Twitter
2021-02-26 09:04:00,"Agenda do dia: tele da Vale (VALE3), de frigoríficos e PIB brasileiro",Suno
2022-02-24 15:23:45,"Petrobras monitora em detalhe evolução da crise na Ucrânia, diz diretor de comercialização\nhttps://t.co/n9u8vLIzpa",Twitter
2022-04-15 19:01:34,Títulos ESG são atropelados por liquidação no mercado global\n#investimentos #mercadofinanceiro #leiamoneytimes\nhttps://t.co/2Ly53k6yig,Twitter


In [5]:
df_full.source.value_counts()

Twitter    19985
Suno        2650
Name: source, dtype: int64

#### Preprocessing Data

In [6]:
def remove_emojis(sentence):

    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def remove_valores(sentence):
    new_sentece = ''
    
    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)
        
    return new_sentece

def remove_links(sentence):
    new_sentece = ''
    
    for token in sentence.split():
        if token.startswith('http'):
            token = ''
        new_sentece += ' {}'.format(token)
        
    return new_sentece

In [7]:
# 1. Aplicar preprocessamento nos títulos e textos completos

def apply_preprocessing(df_input: pd.DataFrame) -> pd.DataFrame:
    
    df = df_input.copy()

    # Substituir símbolos importantes
    df['title'] = df['title'].map(lambda s: s.replace('-feira', ''))
    df['title'] = df['title'].map(lambda s: s.replace('-alvo', ' alvo'))
    df['title'] = df['title'].map(lambda s: s.replace('\n', ' '))  
    df['title'] = df['title'].map(lambda s: s.replace('+', ''))
    df['title'] = df['title'].map(lambda s: s.replace('º', ''))
    df['title'] = df['title'].map(lambda s: s.replace("‘", ''))
    df['title'] = df['title'].map(lambda s: s.replace("’", ''))
    df['title'] = df['title'].map(lambda s: s.replace("•", ''))
    df['title'] = df['title'].map(lambda s: s.replace('-', ''))
    df['title'] = df['title'].map(lambda s: s.replace('%', ' por cento'))
    df['title'] = df['title'].map(lambda s: s.replace('R$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('U$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('US$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('S&P 500', 'spx'))
    df['title'] = df['title'].map(lambda s: s.replace('/', '@'))

    # Remove Links, Hashtags e Menções
    df['title'] = df['title'].map(lambda s: remove_links(s))
    df['title'] = df['title'].str.replace('(\#\w+.*?)',"")
    df['title'] = df['title'].str.replace('(\@\w+.*?)',"")

    # Transformar em String e Letras Minúsculas nas Mensagens
    df['title'] = df['title'].map(lambda s: str(s).lower())

    # Remover Pontuações
    df['title'] = df['title'].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis     
    df['title'] = df['title'].map(lambda s: remove_emojis(s))

    # Quebras de Linha desnecessárias
    df['title'] = df['title'].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df['title'] = df['title'].map(lambda s: s.replace('\"', ''))
    df['title'] = df['title'].map(lambda s: s.replace('“', ''))
    df['title'] = df['title'].map(lambda s: s.replace('”', ''))

    # Remover valores
    df['title'] = df['title'].map(lambda s: remove_valores(s))

    # Espaços desnecessários
    df['title'] = df['title'].map(lambda s: s.strip())

    return df

In [8]:
df_full_preprocessed = apply_preprocessing(df_full)
df_full_preprocessed.sample(10)

,title,source
date,,
2022-02-22 18:47:54,petróleo sobe forte com sanções à rússia analistas veem barril em <NUM> dígitos no curto prazo,Twitter
2022-04-06 17:58:28,senado aprova indicação de renato dias de brito gomes para diretoria do bc,Twitter
2021-09-30 15:58:00,câmara aprova auxílio gás para famílias de baixa renda,Suno
2022-04-27 14:27:06,inflação em escalada chega nos mais ricos e encolhe renda mensal,Twitter
2022-03-23 09:25:29,china suspende importação de carne bovina da unidade da jbs em mozarlândia,Twitter
2020-09-22 08:50:00,petrobras petr4 finaliza oferta de recompra de títulos,Suno
2022-03-23 12:25:00,preço do potássio principal insumo usado como fertilizante triplica durante a guerra,Twitter
2022-01-23 09:51:01,magalu mglu3 e varejistas disparam ação da vale vale3 está barata veja as <NUM> notícias mais lidas,Twitter
2022-02-28 15:04:08,representantes de rússia e ucrânia encerram negociação para consultar lideranças locais,Twitter


In [9]:
df_full.loc['2022-03-14 12:56:13']

,title,source
date,,
2022-03-14 12:56:13,"Biscoitos, pães e macarrão deverão ficar mais caros nas próximas semanas, diz Abimapi\nhttps://t.co/LWj9alRPYy",Twitter


---

#### Create Train/Test Splits

### Creating Labelling Functions (LFs)

In [10]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

categories = [POSITIVE, NEGATIVE, ABSTAIN]

#### Positive Sentiment LFs

- Adjectives
- Verbs

In [18]:
@labeling_function()
def lf_news_good_adjs(x):
    with open('./dicts/final/pos_adj.txt') as file:
        adjectives = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in adjectives:
            return POSITIVE
    return ABSTAIN

@labeling_function()
def lf_happiness_words(x):
    with open('./dicts_emocoes/alegria.txt') as f_words_happiness:
        hapiness_words = [line.rstrip() for line in f_words_happiness]
    
    for word in x.title.lower().split():
        if word in hapiness_words:
            return POSITIVE
    return ABSTAIN

@labeling_function()
def lf_news_good_verbs(x):
    with open('./dicts/pos_verbs.txt') as file:
        verbs = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in verbs:
            return POSITIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_dividendos(x):
    dividend_pattern = r".*pag.*dividendo.*|.*anunc.*dividendo.*|.*distrib.*dividendo.*"
    return POSITIVE if re.search(dividend_pattern, x.title.lower(), flags=re.I) else ABSTAIN

@labeling_function()
def lf_regex_resultado_positivo(x):
    raise_pattern = r"fech.*alta.*|.*abr.*alta.*|.*fech.*pos.*|.*abr.*pos.*|.*estre.*alta.*|.*prev.*alta.*|.*result.*positivo.*" 
    return POSITIVE if re.search(raise_pattern, x.title.lower(), flags=re.I) else ABSTAIN

#### Negative Sentiment LFs

In [19]:
@labeling_function()
def lf_news_bad_adjs(x):
    with open('./dicts/final/neg_adj.txt') as f_adj_neg:
        adjectives = [line.rstrip() for line in f_adj_neg]

    for word in x.title.lower().split():
        if word in adjectives:
            return NEGATIVE

    return ABSTAIN

@labeling_function()
def lf_sadness_words(x):
    with open('./dicts_emocoes/tristeza.txt') as f_words_sadness:
        sadness_words = [line.rstrip() for line in f_words_sadness]
    
    for word in x.title.lower().split():
        if word in sadness_words:
            return NEGATIVE

    return ABSTAIN

@labeling_function()
def lf_news_bad_verbs(x):
    with open('./dicts/neg_verbs.txt') as f_verb_neg:
        verbs = [line.rstrip() for line in f_verb_neg]
    
    for word in x.title.lower().split():
        if word in verbs:
            return NEGATIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_resultado_negativo(x):
    fall_pattern = r"fech.*queda.*|.*abr.*queda.*|.*fech.*neg.*|.*abr.*neg.*|.*prev.*baixa.*|.*prev.*queda.*|.*em.*queda.*|.*result.*negativo.*" 
    return NEGATIVE if re.search(fall_pattern, x.title.lower(), flags=re.I) else ABSTAIN

In [24]:
# combine all the labeling functions 

df = df_full_preprocessed.copy()

df = df[['title']]

lfs = [
       lf_news_good_adjs,
       lf_happiness_words,
       lf_news_good_verbs,
       lf_regex_dividendos,
       lf_regex_resultado_positivo,
       lf_news_bad_adjs,
       lf_sadness_words,
       lf_news_bad_verbs,
       lf_regex_resultado_negativo
]


# apply the label model
applier = PandasLFApplier(lfs=lfs)

label_model = LabelModel(cardinality=len(categories),
                         device='cpu', 
                         verbose=False)

# apply the lfs on the dataframe
L_train = applier.apply(df=df, 
                        progress_bar=False)

# fit on the data
label_model.fit(L_train,
                n_epochs=500,
                log_freq=100, 
                seed=123)

# predict and create the labels
df['label'] = label_model.predict(L=L_train, 
                                  tie_break_policy='abstain').astype(str)

# Convert Labels to Real classes
dict_map = {'-1': 'NEUTRAL', '1': 'POSITIVE', '0': 'NEGATIVE'}
df['label_class'] = df['label'].map(dict_map)

100%|██████████| 500/500 [00:00<00:00, 1758.87epoch/s]


In [25]:
# Polarity: The set of unique labels this LF outputs (excluding abstains)
# Coverage: The fraction of the dataset that each LF labels
# Overlaps: The fraction of the dataset where this LF and at least one other LF label
# Conflicts: The fraction of the dataset where this LF and at least one other LF label and disagree

LFAnalysis(L=L_train, lfs=lfs).lf_summary()#[['Coverage', 'Overlaps', 'Conflicts']] * 100

,j,Polarity,Coverage,Overlaps,Conflicts
lf_news_good_adjs,0,[1],0.148045,0.066534,0.021736
lf_happiness_words,1,[1],0.158648,0.072631,0.027347
lf_news_good_verbs,2,[1],0.048818,0.020676,0.008129
lf_regex_dividendos,3,[1],0.016479,0.009101,0.000928
lf_regex_resultado_positivo,4,[1],0.021913,0.009278,0.003623
lf_news_bad_adjs,5,[0],0.087784,0.041131,0.030881
lf_sadness_words,6,[0],0.033090,0.021206,0.011133
lf_news_bad_verbs,7,[0],0.015065,0.006848,0.004285
lf_regex_resultado_negativo,8,[0],0.029291,0.012194,0.010426


In [26]:
df[['title', 'label_class']].sample(10)

,title,label_class
date,,
2022-04-19 20:18:31,ark invest espera que ações da tesla mais que quadrupliquem,NEUTRAL
2022-04-26 19:53:35,via viia3 elege presidente da wework américa latina para conselho de administração,NEUTRAL
2022-04-28 13:24:33,ganhou <NUM> na poupança pois teria lucrado mais que o dobro nesta renda fixa turbinada com garantia do fgc entenda,POSITIVE
2021-08-06 13:46:00,petrobras petr4 bancos e analistas elevam preço alvo após balanço surpreendente,NEUTRAL
2019-08-08 10:35:00,banco do brasil registra lucro líquido ajustado de <NUM> bi alta de <NUM> por cento,POSITIVE
2021-02-01 13:18:27,o de hoje foi da espaçolaser espa3 a primeira empresa de serviços de beleza a abrir capital por aqui estaremos juntos nesta nova fase de sucesso espaçolaser a b3 agora também é a casa de vocês,POSITIVE
2022-03-07 06:37:16,mês do consumidor mutirão para renegociar dívidas com bancos começa hoje,NEUTRAL
2022-04-20 08:29:58,vale usiminas netflix heineken assaí e mais veja os destaques das empresas,NEUTRAL
2022-04-04 13:33:35,análises mistas sobre números do primeiro trimestre da tesla,NEUTRAL


In [27]:
#Filtering out unlabeled data points
df = df.loc[df.label_class.isin(['POSITIVE', 'NEGATIVE']), :]

print ('Quantidade Total de Mensagens Rotuladas: ', df.shape[0])

# find the label counts 
df['label_class'].value_counts() / df.shape[0] * 100

Quantidade Total de Mensagens Rotuladas:  9445


POSITIVE    74.219164
NEGATIVE    25.780836
Name: label_class, dtype: float64

In [28]:
df.sample(10)

,title,label,label_class
date,,,
2022-04-18 05:00:01,reta final da alta da selic falta pouco para saber como fisgar as oportunidades do novo cenário para ajudar você hoje <NUM> o repórter recebe às 9h paloma brum analista da toro investimentos e rodrigo galindo gestora da novus capital,0,NEGATIVE
2022-03-31 18:09:11,mxrf11 sobe <NUM> por cento e puxa altas do ifix no último pregão de março índice sobe <NUM> por cento no mês,0,NEGATIVE
2022-04-22 19:16:35,<NUM> etfs para investir na recémcriada warner bros discovery,1,POSITIVE
2022-03-01 18:36:09,índice de adrs brazil titans titubeia no fim do pregão mas fecha em alta de <NUM> por cento papéis da petrobras lideram altas,1,POSITIVE
2022-04-01 12:13:44,deputados têm até hoje para trocar de partido sem perder mandato,0,NEGATIVE
2022-03-04 21:39:03,dúvidas sobre investir na bolsa não tema pois o atende b3 está aqui confira o e saiba mais sobre o nosso canal pra ter respostas,0,NEGATIVE
2022-04-20 15:23:36,aura minerals aura33 como a aquisição da big river gold pode afetar dividendos,1,POSITIVE
2022-03-15 10:06:24,falta de qualificação é principal entrave para empregar jovens diz pesquisa,0,NEGATIVE
2022-01-22 10:05:02,magazine luiza mglu3 via viia3 quais são as previsões do ecommerce para <NUM>,1,POSITIVE
